In [26]:
import os
import re
import numpy as np
import pandas as pd
import unicodedata
from datetime import datetime
import unicodedata

In [27]:
# Ruta del archivo csv 
FILES_PATH = os.path.join('..','..','data','raw','GeekyMotors')
csv_path = os.path.join(FILES_PATH, 'geeky_motors2.csv')

with open(csv_path, 'r', encoding='latin1') as f:
    data = pd.read_csv(f)
    
data

,nombre,kilometraje,estado,combustible,carroceria,traccion,transmision,asientos,color,verificacion,precio
0,Tesla Model 3 2018 Bajo Perfil,44000,Usado,Eléctrico,Sedan,4x4,Automática,Cuero vegano,Red Multi-Coat,93.0,"$55,000"
1,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
2,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
3,Toyota Corolla Sport Edition Full Extras 2019,78000,Usado,Gasolina,Sedan,4x2,Automática,Cuero,Azul Marino,95.0,"$17,500"
4,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
...,...,...,...,...,...,...,...,...,...,...,...
4283,Nissan Almera 2005,150000,Usado,Gasolina,Sedan,4x2,Automática,Tela,Rojo Vino,NaN,"$3,500"
4284,Kia Sportage full xtra 2017 Negociable,96000,Usado,Gasolina,SUV,4x2,Automática,"Tela, oscuros y áreas de cuero",Negro / gris,NaN,"$12,999"
4285,BMW X5 3.OD 2014,167000,Usado,Diesel,SUV,4x4,Automática,cuero,crema,NaN,"$18,000"
4286,Jeep Wrangler Unlimited 2018,78000,Usado,Gasolina,SUV,4x4,Automática,Tela,Beige,NaN,"$25,000"


In [28]:
def normalizar_nombres_columnas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza los nombres de las columnas de un DataFrame.
    Convierte los nombres a minúsculas y elimina las tildes.

    Parameters:
    - df: pd.DataFrame
        El DataFrame cuyas columnas se normalizarán.

    Retorna:
    - pd.DataFrame
        El DataFrame con los nombres de columnas normalizados.
    """
    def remover_tildes(texto):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        return ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')

    # Normalizar los nombres de las columnas
    df.columns = [remover_tildes(col).lower() for col in df.columns]
    
    return df


In [29]:
data = normalizar_nombres_columnas(data)
data.columns

Index(['nombre', 'kilometraje', 'estado', 'combustible', 'carroceria',
       'traccion', 'transmision', 'asientos', 'color', 'verificacion',
       'precio'],
      dtype='object')

In [30]:
for col in data.columns:
    if data[col].nunique() < 20:
        print(col)
        print(data[col].unique())
        print()

estado
['Usado']

combustible
['Eléctrico' 'Gasolina' 'Diesel']

carroceria
['Sedan' 'SUV' 'Pickup' 'Coupe' 'Hatchback' 'Convertible' 'Wagon']

traccion
['4x4' '4x2']

transmision
['Automática' 'Manual']

asientos
['Cuero vegano' 'Cuerro' 'Cuero' 'Algodón' 'Tela' 'cuero suave' 'cuero'
 'CUERO' 'tela' 'Asientos de tela con forros de cuerinq' 'Tela, off road'
 'Cuerina Premium' 'TELA' 'Cuero rojo con negro' 'Asientos de tela' '5'
 'Cuerina' 'Tela, oscuros y áreas de cuero']

verificacion
[ 93.  nan  95.  92. 100.  98.]



### Marca

In [31]:
def estandarizar_marcas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función estandariza los nombres de marcas en una columna de un DataFrame de pandas
    según un diccionario de reemplazos proporcionado. Los nombres de marcas en la columna se
    reemplazan con claves específicas del diccionario si se encuentran las variantes especificadas.

    Parameters:
    - df: pd.DataFrame
        DataFrame que debe contener una columna llamada 'Nombre' con los nombres de las marcas.

    Retorna:
    - pd.DataFrame
        Un DataFrame con la columna 'Nombre' actualizada con las marcas estandarizadas.
    """

    reemplazos_marcas = {
        'Hyundai': ['hyudani'],
        'Porsche': ['porche'],
        'Mercedes-Benz': ['mercedes']
    }

    # Iterar sobre el diccionario de reemplazos
    for clave, valores in reemplazos_marcas.items():
        for valor in valores:
            # Reemplaza cada variante en la columna con la marca estandarizada
            df['nombre'] = df['nombre'].str.replace(valor.lower(), clave, case=False, regex=False)
            
    return df


In [32]:
data = estandarizar_marcas(data)
data

,nombre,kilometraje,estado,combustible,carroceria,traccion,transmision,asientos,color,verificacion,precio
0,Tesla Model 3 2018 Bajo Perfil,44000,Usado,Eléctrico,Sedan,4x4,Automática,Cuero vegano,Red Multi-Coat,93.0,"$55,000"
1,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
2,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
3,Toyota Corolla Sport Edition Full Extras 2019,78000,Usado,Gasolina,Sedan,4x2,Automática,Cuero,Azul Marino,95.0,"$17,500"
4,Audi Q5 2012,154000,Usado,Gasolina,SUV,4x2,Automática,Cuerro,Gris,NaN,"$6,500"
...,...,...,...,...,...,...,...,...,...,...,...
4283,Nissan Almera 2005,150000,Usado,Gasolina,Sedan,4x2,Automática,Tela,Rojo Vino,NaN,"$3,500"
4284,Kia Sportage full xtra 2017 Negociable,96000,Usado,Gasolina,SUV,4x2,Automática,"Tela, oscuros y áreas de cuero",Negro / gris,NaN,"$12,999"
4285,BMW X5 3.OD 2014,167000,Usado,Diesel,SUV,4x4,Automática,cuero,crema,NaN,"$18,000"
4286,Jeep Wrangler Unlimited 2018,78000,Usado,Gasolina,SUV,4x4,Automática,Tela,Beige,NaN,"$25,000"


In [33]:
def extraer_marca(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función extrae las marcas de autos y las agrega a una nueva columna en el DataFrame.

    Parameters:
    - df: pd.DataFrame
        DataFrame que debe contener una columna llamada 'Nombre' con los nombres de los autos.

    Retorna:
    - pd.DataFrame
        DataFrame con una nueva columna llamada 'marca' que contiene la marca extraída.
    """
    # Lista de marcas de autos
    lista_marcas = [
        "acura", "alfa romeo", "aston martin", "audi", "bentley", "bmw", "bugatti", "buick",
        "cadillac", "chevrolet", "chrysler", "citroën", "dacia", "daewoo", "daihatsu", "dodge",
        "ferrari", "fiat", "ford", "geely", "genesis", "gmc", "honda", "hummer", "hyundai", "infiniti",
        "isuzu", "jaguar", "jeep", "kia", "koenigsegg", "lamborghini", "lancia", "land rover",
        "lexus", "lincoln", "lotus", "mahindra", "maserati", "maybach", "mazda", "mclaren", "mercedes-benz",
        "mercury", "mg", "mini", "mitsubishi", "morgan", "nissan", "opel", "pagani", "peugeot",
        "plymouth", "polestar", "pontiac", "porsche", "ram", "renault", "rolls-royce", "saab",
        "seat", "škoda", "smart", "subaru", "suzuki", "tata", "tesla", "toyota", "volkswagen",
        "volvo", "wiesmann", "zotye", "byd", "chery", "jac", "ssangyong", "great wall", "fisker", "rivian", "lucid", "vinfast"
    ]

    def encontrar_marca(texto):
        texto = texto.lower()
        for marca in lista_marcas:
            patron = r'\b' + re.escape(marca) + r'\b'
            if re.search(patron, texto):
                return marca
        return "Desconocida"  # Retorna un valor por defecto si no se encuentra ninguna marca

    df['marca'] = df['nombre'].apply(encontrar_marca)
    return df


In [34]:
data = extraer_marca(data)
data.marca.unique()

array(['tesla', 'audi', 'toyota', 'nissan', 'hyundai', 'kia', 'bmw',
       'honda', 'mini', 'mercedes-benz', 'subaru', 'suzuki', 'renault',
       'mazda', 'acura', 'geely', 'lincoln', 'land rover', 'ford',
       'maserati', 'mitsubishi', 'mahindra', 'peugeot', 'isuzu',
       'volkswagen', 'chevrolet', 'jeep', 'dodge', 'porsche'],
      dtype=object)

### Año

In [35]:
def extraer_anio(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función extrae los años del modelo de los autos de una columna de nombres de autos
    y los agrega como una nueva columna en el DataFrame.

    Parameters:
    - df: pd.DataFrame
        DataFrame que debe contener una columna llamada 'Nombre' con los nombres de los autos.

    Retorna:
    - pd.DataFrame
        Un DataFrame con una nueva columna llamada 'anio' que contiene los años extraídos.
    """

    # Lista de años posibles
    lista_anios = np.arange(1990, datetime.now().year + 1)
    
    def encontrar_anio(nombre):
        for anio in lista_anios:
            if str(anio) in nombre:
                return anio
        return np.nan  # Retorna NaN si no se encuentra un año

    # Aplicar la función a cada fila de la columna 'Nombre'
    df['anio'] = df['nombre'].apply(encontrar_anio)
    
    return df


In [36]:
data = extraer_anio(data)
data.anio.unique()

array([2018, 2012, 2019, 2015, 2017, 2013, 2016, 2020, 2022, 2023, 2021,
       2010, 2005, 2014, 1997, 2024, 1993, 2004, 2000])

### Modelo

In [37]:
def extraer_modelo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función extrae los modelos de autos de una columna de nombres de autos,
    utilizando las columnas de marca y año como referencia.

    Parameters:
    - df: pd.DataFrame
        DataFrame que debe contener las columnas 'Nombre', 'marca' y 'anio'.

    Retorna:
    - pd.DataFrame
        Un DataFrame con una nueva columna llamada 'modelo' que contiene los modelos extraídos.
    """
    
    def encontrar_modelo(fila):
        nombre_auto = fila['nombre']
        marca = fila['marca']
        anio = fila['anio']
        
        # Encontrar la posición de la marca y el año en el texto
        start_index = nombre_auto.lower().find(marca.lower()) + len(marca)
        end_index = nombre_auto.find(str(anio))

        if start_index != -1 and end_index != -1 and end_index > start_index:
            # Extraer el texto entre la marca y el año como modelo
            modelo = nombre_auto[start_index:end_index].strip()
            return modelo
        return ""  # Retorna cadena vacía si no se encuentra modelo

    # Aplicar la función para extraer modelo a cada fila
    df['modelo'] = df.apply(encontrar_modelo, axis=1)
    
    return df


In [38]:
data = extraer_modelo(data)
data.modelo.unique()

array(['Model 3', 'Q5', 'Corolla Sport Edition Full Extras', 'Navara',
       'i20-Active', 'Extrail', 'RIO', 'Cerato', '218i Gran Coupï¿½',
       'Civic full extra', 'Cooper',
       '-Benz Mercedes-Benz AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson',
       'Model X', 'Xtrail', 'Captur', '318i', 'Crv', 'Q3', 'Accent',
       'Corolla', 'X-trail', 'CX-9', 'Tucson GLS Full', 'Land Cruiser',
       'Rogue', 'TLX', '-Benz GLE 400', 'Grand i10', '650i', 'GLC 200',
       'LAND CRUISER Prado', 'EON', 'Civic', 'Swift dzire', 'Coolray',
       'Outback', 'K2700', 'Aviator', 'CR-V', 'Sportage Gt Line',
       'Tundra', 'Rav 4', 'Yaris', 'MDX',
       'Range Rover Vogue supercharged', 'Range Rover Sport', 'Cx5',
       'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1',
       'Levante S Negociable', 'HILUX', 'F-150 Raptor', '640',
       'RIO HATCHBACK', 'Yaris S FULL', 'Asx', 'Vitara', 'Figo', '',
       'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT', 'COROLLA SE',
       'Pik Up 2.5', 'CRETA',

In [39]:
def remover_textos(df:pd.DataFrame) -> pd.DataFrame:
    """
    Esta función remueve textos indeseados como: 'full extra', etc, de los modelos de autos.

    Parameters:
    - df: pd.DataFrame
        Dataframe que debe contener la columna 'modelo'.
    
    Retorna:
    - pd.Series
        Una serie con los modelos.
    """

    # Lista de textos a eliminar
    textos_a_eliminar = ['full', 'xtra', 'extra', 'extras', 'full extra', 'full extras', 'full xtra', 'hatchback', 'negociable', 'mercedes']

    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(texto) for texto in textos_a_eliminar) + r')\b', re.IGNORECASE)
    df['modelo'] = df['modelo'].apply(lambda x: pattern.sub('', str(x)).strip())

    return df

In [40]:
data = remover_textos(data)
data.modelo.unique()

array(['Model 3', 'Q5', 'Corolla Sport Edition', 'Navara', 'i20-Active',
       'Extrail', 'RIO', 'Cerato', '218i Gran Coupï¿½', 'Civic', 'Cooper',
       '-Benz -Benz AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X',
       'Xtrail', 'Captur', '318i', 'Crv', 'Q3', 'Accent', 'Corolla',
       'X-trail', 'CX-9', 'Tucson GLS', 'Land Cruiser', 'Rogue', 'TLX',
       '-Benz GLE 400', 'Grand i10', '650i', 'GLC 200',
       'LAND CRUISER Prado', 'EON', 'Swift dzire', 'Coolray', 'Outback',
       'K2700', 'Aviator', 'CR-V', 'Sportage Gt Line', 'Tundra', 'Rav 4',
       'Yaris', 'MDX', 'Range Rover Vogue supercharged',
       'Range Rover Sport', 'Cx5', 'tercel', 'A3', 'F150 Lariat', 'Kicks',
       'Venue', 'X1', 'Levante S', 'HILUX', 'F-150 Raptor', '640',
       'Yaris S', 'Asx', 'Vitara', 'Figo', '', 'Q3 Sportback',
       'Ranger XL', 'BRONCO SPORT', 'COROLLA SE', 'Pik Up 2.5', 'CRETA',
       '301', 'ILX', 'ISUZU', 'TIGUAN', 'Traverse',
       'Grand Cherokee Laredo', 'Ford EDGE SEL

In [41]:
def transformar_sport_edition(df:pd.DataFrame) -> pd.DataFrame:
    """
    Convierte el texto 'Sport Edition' a 'SE'.

    Parameters:
    - df: pd.DataFrame
        Dataframe que debe contener la columna 'modelo'.

    Retorna:
    - pd.DataFrame
        Una serie con los modelos.
    """

    pattern = re.compile(r'\bsport[\s-]*edition\b', re.IGNORECASE)
    df['modelo'] = df['modelo'].apply(lambda x: pattern.sub('SE', str(x)))

    return df

In [42]:
data = transformar_sport_edition(data)
data.modelo.unique()


array(['Model 3', 'Q5', 'Corolla SE', 'Navara', 'i20-Active', 'Extrail',
       'RIO', 'Cerato', '218i Gran Coupï¿½', 'Civic', 'Cooper',
       '-Benz -Benz AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X',
       'Xtrail', 'Captur', '318i', 'Crv', 'Q3', 'Accent', 'Corolla',
       'X-trail', 'CX-9', 'Tucson GLS', 'Land Cruiser', 'Rogue', 'TLX',
       '-Benz GLE 400', 'Grand i10', '650i', 'GLC 200',
       'LAND CRUISER Prado', 'EON', 'Swift dzire', 'Coolray', 'Outback',
       'K2700', 'Aviator', 'CR-V', 'Sportage Gt Line', 'Tundra', 'Rav 4',
       'Yaris', 'MDX', 'Range Rover Vogue supercharged',
       'Range Rover Sport', 'Cx5', 'tercel', 'A3', 'F150 Lariat', 'Kicks',
       'Venue', 'X1', 'Levante S', 'HILUX', 'F-150 Raptor', '640',
       'Yaris S', 'Asx', 'Vitara', 'Figo', '', 'Q3 Sportback',
       'Ranger XL', 'BRONCO SPORT', 'COROLLA SE', 'Pik Up 2.5', 'CRETA',
       '301', 'ILX', 'ISUZU', 'TIGUAN', 'Traverse',
       'Grand Cherokee Laredo', 'Ford EDGE SEL', 'Cx-5', 

In [43]:
def estandarizar_modelos(df:pd.DataFrame):
    """
    Estandariza los modelos, reemplazando los textos mal digitados.

    Parameters:
    - df: pd.DataFrame
        Dataframe que debe contener la columna 'modelo'.

    Retorna:
    - pd.Dataframe
        Una serie con los modelos.
    """
    reemplazos_modelos = {'X-Trail': ['extrail','xtrail','X-trail'], 'CRV':['Crv','cr-v'], 'CX-5':['Cx-5','cx5'], 'Rio':['RIO'], 'CX-9':['cx9'],
              'Pick Up':['pik up']}
    
    # Iterar sobre el diccionario de reemplazos
    for clave, valores in reemplazos_modelos.items():
        for valor in valores:
            # Reemplaza cada valor en la columna temporal en minúsculas
            df['modelo'] = df['modelo'].str.replace(rf'{valor}', clave, case=False, regex=False)

    return df


In [44]:
data = estandarizar_modelos(data)
data.modelo.unique()


array(['Model 3', 'Q5', 'Corolla SE', 'Navara', 'i20-Active', 'X-Trail',
       'Rio', 'Cerato', '218i Gran Coupï¿½', 'Civic', 'Cooper',
       '-Benz -Benz AMG GLE 63s Coupe', 'XV', 'Ciaz', 'Tucson', 'Model X',
       'Captur', '318i', 'CRV', 'Q3', 'Accent', 'Corolla', 'CX-9',
       'Tucson GLS', 'Land Cruiser', 'Rogue', 'TLX', '-Benz GLE 400',
       'Grand i10', '650i', 'GLC 200', 'LAND CRUISER Prado', 'EON',
       'Swift dzire', 'Coolray', 'Outback', 'K2700', 'Aviator',
       'Sportage Gt Line', 'Tundra', 'Rav 4', 'Yaris', 'MDX',
       'Range Rover Vogue supercharged', 'Range Rover Sport', 'CX-5',
       'tercel', 'A3', 'F150 Lariat', 'Kicks', 'Venue', 'X1', 'Levante S',
       'HILUX', 'F-150 Raptor', '640', 'Yaris S', 'Asx', 'Vitara', 'Figo',
       '', 'Q3 Sportback', 'Ranger XL', 'BRONCO SPORT', 'COROLLA SE',
       'Pick Up 2.5', 'CRETA', '301', 'ILX', 'ISUZU', 'TIGUAN',
       'Traverse', 'Grand Cherokee Laredo', 'Ford EDGE SEL', 'WRANGLER',
       'Range Rover Evoque', '

### Resto de variables

In [45]:
def normalizar_valores_columnas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza los valores de las columnas de un DataFrame.
    Convierte los valores de tipo texto a minúsculas y elimina las tildes.

    Parameters:
    - df: pd.DataFrame
        El DataFrame cuyos valores se normalizarán.

    Retorna:
    - pd.DataFrame
        El DataFrame con los valores de las columnas de texto normalizados.
    """
    def remover_tildes(texto):
        if isinstance(texto, str):
            texto_normalizado = unicodedata.normalize('NFD', texto)
            return ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn').lower()
        return texto  # Si no es string, devolver el valor original

    # Aplicar la normalización solo a las columnas de tipo 'object' (texto)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(remover_tildes)
    
    return df

In [46]:
data = normalizar_valores_columnas(data)
data

,nombre,kilometraje,estado,combustible,carroceria,traccion,transmision,asientos,color,verificacion,precio,marca,anio,modelo
0,tesla model 3 2018 bajo perfil,44000,usado,electrico,sedan,4x4,automatica,cuero vegano,red multi-coat,93.0,"$55,000",tesla,2018,model 3
1,audi q5 2012,154000,usado,gasolina,suv,4x2,automatica,cuerro,gris,NaN,"$6,500",audi,2012,q5
2,audi q5 2012,154000,usado,gasolina,suv,4x2,automatica,cuerro,gris,NaN,"$6,500",audi,2012,q5
3,toyota corolla sport edition full extras 2019,78000,usado,gasolina,sedan,4x2,automatica,cuero,azul marino,95.0,"$17,500",toyota,2019,corolla se
4,audi q5 2012,154000,usado,gasolina,suv,4x2,automatica,cuerro,gris,NaN,"$6,500",audi,2012,q5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4283,nissan almera 2005,150000,usado,gasolina,sedan,4x2,automatica,tela,rojo vino,NaN,"$3,500",nissan,2005,almera
4284,kia sportage full xtra 2017 negociable,96000,usado,gasolina,suv,4x2,automatica,"tela, oscuros y areas de cuero",negro / gris,NaN,"$12,999",kia,2017,sportage
4285,bmw x5 3.od 2014,167000,usado,diesel,suv,4x4,automatica,cuero,crema,NaN,"$18,000",bmw,2014,x5 3.od
4286,jeep wrangler unlimited 2018,78000,usado,gasolina,suv,4x4,automatica,tela,beige,NaN,"$25,000",jeep,2018,wrangler unlimited


### Precio

In [47]:
def transformar_precio(df: pd.Series) -> pd.Series:
    """
    Convierte el precio a tipo int.

    Parameters:
    - df: pd.DataFrame
        Dataframe que debe contener la columna 'Precio'.
    
    Retorna:
    - pd.Dataframe
        Dataframe con la columna 'precio' en tipo 'int'.
    """
    df['precio'] = df['precio'].str.replace('$','').str.replace(',','')
    df['precio'] = df['precio'].astype(int)

    return df

In [48]:
data = transformar_precio(data)
data.precio

0       55000
1        6500
2        6500
3       17500
4        6500
        ...  
4283     3500
4284    12999
4285    18000
4286    25000
4287    34995
Name: precio, Length: 4288, dtype: int64

### Final

In [49]:
def selector_columnas(df:pd.DataFrame) -> pd.DataFrame:
    mis_variables = ['marca','modelo','anio','carroceria','traccion','transmision',
                     'asientos','color','kilometraje','verificacion','precio']
    
    return df[mis_variables]

In [50]:
selector_columnas(data)

,marca,modelo,anio,carroceria,traccion,transmision,asientos,color,kilometraje,verificacion,precio
0,tesla,model 3,2018,sedan,4x4,automatica,cuero vegano,red multi-coat,44000,93.0,55000
1,audi,q5,2012,suv,4x2,automatica,cuerro,gris,154000,NaN,6500
2,audi,q5,2012,suv,4x2,automatica,cuerro,gris,154000,NaN,6500
3,toyota,corolla se,2019,sedan,4x2,automatica,cuero,azul marino,78000,95.0,17500
4,audi,q5,2012,suv,4x2,automatica,cuerro,gris,154000,NaN,6500
...,...,...,...,...,...,...,...,...,...,...,...
4283,nissan,almera,2005,sedan,4x2,automatica,tela,rojo vino,150000,NaN,3500
4284,kia,sportage,2017,suv,4x2,automatica,"tela, oscuros y areas de cuero",negro / gris,96000,NaN,12999
4285,bmw,x5 3.od,2014,suv,4x4,automatica,cuero,crema,167000,NaN,18000
4286,jeep,wrangler unlimited,2018,suv,4x4,automatica,tela,beige,78000,NaN,25000
